In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [15]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [16]:
## Defino funciones dimension mercado de trabajo e ingresos

# Componente: Inserción laboral

# T1. 
# Variable: Actividad productiva
# Indicador: Ratio M/V en tasa de empleo

def actividad_productiva(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de empleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de empleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de empleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de empleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [17]:
tasa, size, ratio, error = actividad_productiva(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo = pd.concat([tasa, size, ratio, error], axis=1)
empleo

,Varon,Mujer,N_v,N_m,Actividad productiva,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,78.710304,70.957452,642,594,90.150144,85.317535,94.982752,4.832609,5.360622
6,75.153672,55.778992,3985,3250,74.219915,72.048389,76.391442,2.171526,2.925800
10,69.529090,52.433530,1026,813,75.412363,70.971985,79.852740,4.440377,5.888129
14,79.622598,54.541412,1906,1487,68.499915,65.389416,71.610414,3.110499,4.540880
18,72.778626,48.508609,924,661,66.652274,61.882320,71.422227,4.769953,7.156475
22,73.566052,44.943799,829,608,61.093124,56.128739,66.057509,4.964385,8.125930
26,73.485409,55.669633,1176,973,75.756036,71.742573,79.769499,4.013463,5.297879
30,73.334174,51.223948,1496,1100,69.850038,66.142374,73.557702,3.707664,5.308034
34,64.272893,32.303294,841,459,50.259593,44.893889,55.625296,5.365703,10.675979


In [18]:
# Componente: Inserción laboral

# T2. 
# Variable: Busqueda de trabajo
# Indicador: Ratio M/V en tasa de desempleo

def busqueda_trabajo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de desempleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de desempleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de desempleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de desempleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    df_estado = df_temp[(df_temp['ESTADO'] == 1) | (df_temp['ESTADO'] == 2)]
    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Búsqueda de trabajo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [19]:
tasa, size, ratio, error = busqueda_trabajo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

desempleo = pd.concat([tasa, size, ratio, error], axis=1)

desempleo

,Varon,Mujer,N_v,N_m,Búsqueda de trabajo,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,6.270256,5.029093,39,36,124.679654,114.246343,135.112966,10.433311,8.368095
6,8.252182,10.440042,301,342,79.043576,74.553036,83.534116,4.490540,5.681094
10,4.901536,8.311059,55,59,58.976074,49.911190,68.040958,9.064884,15.370443
14,6.333984,12.016204,129,164,52.712020,46.198097,59.225943,6.513923,12.357567
18,4.921424,4.541953,43,27,108.354805,98.181874,118.527737,10.172931,9.388537
22,4.864433,4.154527,44,27,117.087538,107.235796,126.939280,9.851742,8.413997
26,4.906585,3.513886,71,43,139.634140,132.182078,147.086201,7.452062,5.336848
30,5.460783,10.521391,94,96,51.901716,44.235584,59.567849,7.666133,14.770480
34,5.816408,2.355315,50,13,246.948214,236.457868,257.438559,10.490346,4.247994


In [20]:
# Componente: Inserción laboral

# T3. 
# Variable: Acceso a derechos laborales
# Indicador: Ratio M/V en tasa de registro de la relación laboral

def acceso_derechos_laborales(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción de tasa de registro de la relación laboral entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de registro (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de registro (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de registro (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07H']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a derechos laborales'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [21]:
tasa, size, ratio, error = acceso_derechos_laborales(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

empleo_asalariado_con_descuento = pd.concat([tasa, size, ratio, error], axis=1)

empleo_asalariado_con_descuento

,Varon,Mujer,N_v,N_m,Acceso a derechos laborales,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,79.665069,76.119022,359,364,95.548805,89.505744,101.591866,6.043061,6.324580
6,64.819435,61.219465,1916,1584,94.446157,91.232178,97.660137,3.213979,3.402975
10,61.784084,60.456728,507,404,97.851622,91.478110,104.225134,6.373512,6.513446
14,54.908631,50.804343,773,598,92.525240,87.199869,97.850611,5.325371,5.755588
18,55.114564,53.627512,357,258,97.301889,89.323982,105.279797,7.977907,8.199129
22,45.381320,56.913699,272,258,125.412171,116.955340,133.869002,8.456831,6.743230
26,79.404827,71.502111,704,574,90.047562,85.297781,94.797343,4.749781,5.274747
30,65.913125,56.866193,639,515,86.274460,80.635040,91.913880,5.639420,6.536604
34,50.218551,57.363368,306,206,114.227444,105.452910,123.001978,8.774534,7.681634


In [22]:
df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas

# Filtrar y contar las combinaciones por género
resultados = df_estado.groupby(['CH04', 'PP07H', 'PP07I'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# Renombrar las columnas y restablecer los índices
resultados.columns = ['Varones', 'Mujeres']
resultados = resultados.reset_index()

resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']

resultados['Combinación'] = resultados.apply(lambda row: f"PP07H == {row['PP07H']} y PP07I == {row['PP07I']}", axis=1)

resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# Aplicar formato a los valores en la tabla
tabla_pob_ocupada_asalariada = resultados.style.format({
    'Varones': '{:,.0f}',
    '% Varones': '{:,.2f}',
    'Mujeres': '{:,.0f}',
    '% Mujeres': '{:,.2f}',
    'Total Fila': '{:,.0f}'
})

tabla_pob_ocupada_asalariada

,Combinación,Varones,% Varones,Mujeres,% Mujeres,Total Fila
0,PP07H == 1.0 y PP07I == 0.0,"4,451,936",63.75,"3,397,674",60.31,"7,849,610"
1,PP07H == 2.0 y PP07I == 0.0,"27,918",0.40,"13,121",0.23,"41,039"
2,PP07H == 2.0 y PP07I == 1.0,"277,412",3.97,"252,810",4.49,"530,222"
3,PP07H == 2.0 y PP07I == 2.0,"2,209,098",31.64,"1,965,206",34.88,"4,174,304"
4,PP07H == 2.0 y PP07I == 9.0,"16,529",0.24,"4,650",0.08,"21,180"
Total Col,nan,"6,982,893",100.00,"5,633,461",100.00,"12,616,354"


In [37]:
# Componente: Inserción laboral

# T4. 
# Variable: Participación en el trabajo remunerado
# Indicador: Ratio M/V en las horas trabajadas remuneradas

def participacion_trabajo_remunerado(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción en las horas trabajadas remuneradas entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las horas trabajadas remuneradas (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular las horas trabajadas remuneradas (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de las horas trabajadas remuneradas (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1)]          # personas ocupadas 

    df_estado['Horas'] = df_estado[['PP3E_TOT', 'PP3F_TOT']].max(axis=1)

#    numerador = df_estado[(df_estado['PP3E_TOT']>0) & (df_estado['PP3E_TOT']!=999)].groupby(['CH04', var])['PP3E_TOT'].sum().unstack(level=0)
    numerador = df_estado[(df_estado['Horas']>0) & (df_estado['Horas']!=999)].groupby(['CH04', var])['Horas'].sum().unstack(level=0)
#    denominador = df_estado[(df_estado['PP3E_TOT']>0) & (df_estado['PP3E_TOT']!=999)].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_estado[(df_estado['Horas']>0) & (df_estado['Horas']!=999)].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[(df_estado['Horas']>0) & (df_estado['Horas']!=999)].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Participacion trabajo remunerado'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [38]:
tasa, size, ratio, error = participacion_trabajo_remunerado(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
horas_remuneradas = pd.concat([tasa, size, ratio, error], axis=1)
horas_remuneradas

,Varon,Mujer,N_v,N_m,Participacion trabajo remunerado,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,3.432555,2.916838,629,568,84.975709,82.990881,86.960538,1.984828,2.335760
6,4.160159,3.458459,3847,3114,83.132875,82.232865,84.032885,0.900010,1.082616
10,54.764366,43.321178,992,766,79.104683,74.424203,83.785163,4.680480,5.916818
14,9.053359,7.711154,1867,1436,85.174507,83.277697,87.071316,1.896810,2.226969
18,17.474204,14.617914,883,625,83.654251,79.919954,87.388547,3.734297,4.463965
22,13.978736,12.744291,824,601,91.169122,87.603508,94.734736,3.565614,3.910988
26,36.093737,31.033598,1133,938,85.980562,81.907989,90.053135,4.072573,4.736621
30,22.276719,16.630567,1465,1069,74.654475,71.568640,77.740311,3.085836,4.133491
34,33.362570,29.491209,830,447,88.396095,83.089540,93.702651,5.306556,6.003156


In [25]:
df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
df_estado = df_temp[(df_temp['ESTADO'] == 1)]          # personas ocupadas 

# Filtrar y contar las combinaciones por género
resultados = df_estado.groupby(['CH04', 'CAT_OCUP', 'PP03C', 'PP03D'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# Renombrar las columnas y restablecer los índices
resultados.columns = ['Varones', 'Mujeres']
resultados = resultados.reset_index()

resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']

resultados['Combinación'] = resultados.apply(lambda row: f"CAT_OCUP == {row['CAT_OCUP']}, PP03C == {row['PP03C']}, PP03D == {row['PP03D']}", axis=1)

resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# Aplicar formato a los valores en la tabla
tabla_pob_ocupada = resultados.style.format({
    'Varones': '{:,.0f}',
    '% Varones': '{:,.2f}',
    'Mujeres': '{:,.0f}',
    '% Mujeres': '{:,.2f}',
    'Total Fila': '{:,.0f}'
})

tabla_pob_ocupada

,Combinación,Varones,% Varones,Mujeres,% Mujeres,Total Fila
0,"CAT_OCUP == 1.0, PP03C == 0.0, PP03D == 0.0","9,305",0.09,"4,898",0.07,"14,203"
1,"CAT_OCUP == 1.0, PP03C == 1.0, PP03D == 0.0","407,295",4.13,"151,377",2.04,"558,672"
2,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 2.0","27,592",0.28,"7,503",0.10,"35,095"
3,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 3.0","2,126",0.02,"4,270",0.06,"6,395"
4,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 5.0",47,0.00,0,0.00,47
5,"CAT_OCUP == 2.0, PP03C == 0.0, PP03D == 0.0","24,561",0.25,"16,184",0.22,"40,744"
6,"CAT_OCUP == 2.0, PP03C == 1.0, PP03D == 0.0","2,224,770",22.56,"1,391,233",18.77,"3,616,002"
7,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 2.0","124,964",1.27,"129,225",1.74,"254,190"
8,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 3.0","6,906",0.07,"20,252",0.27,"27,157"
9,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 4.0",208,0.00,933,0.01,"1,141"


In [36]:
df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] != 1)& (df_temp['CAT_OCUP'] != 4)]          # personas ocupadas 

# Filtrar y contar las combinaciones por género
resultados = df_estado.groupby(['CH04', 'CAT_OCUP', 'PP03C'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# Renombrar las columnas y restablecer los índices
resultados.columns = ['Varones', 'Mujeres']
resultados = resultados.reset_index()

resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']

resultados['Combinación'] = resultados.apply(lambda row: f"CAT_OCUP == {row['CAT_OCUP']}, PP03C == {row['PP03C']}", axis=1)

resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# Aplicar formato a los valores en la tabla
tabla_pob_ocupada = resultados.style.format({
    'Varones': '{:,.0f}',
    '% Varones': '{:,.2f}',
    'Mujeres': '{:,.0f}',
    '% Mujeres': '{:,.2f}',
    'Total Fila': '{:,.0f}'
})

tabla_pob_ocupada

# [P21 mean M y V, TOT_P12 mean M y V]

,Combinación,Varones,% Varones,Mujeres,% Mujeres,Total Fila
0,"CAT_OCUP == 2.0, PP03C == 0.0","24,561",0.26,"16,184",0.23,"40,744"
1,"CAT_OCUP == 2.0, PP03C == 1.0","2,224,770",23.76,"1,391,233",19.34,"3,616,002"
2,"CAT_OCUP == 2.0, PP03C == 2.0","133,009",1.42,"151,403",2.11,"284,412"
3,"CAT_OCUP == 3.0, PP03C == 0.0","223,539",2.39,"262,284",3.65,"485,822"
4,"CAT_OCUP == 3.0, PP03C == 1.0","6,201,214",66.22,"4,571,237",63.56,"10,772,451"
5,"CAT_OCUP == 3.0, PP03C == 2.0","558,140",5.96,"799,941",11.12,"1,358,081"
Total Col,nan,"9,365,232",100.00,"7,192,281",100.00,"16,557,513"


### Analisis dfs

In [26]:
# cod_provincia
dict_cod_provincia = {2: "CABA",
 6: "Buenos Aires",
 10: "Catamarca",
 14: "Córdoba",
 18: "Corrientes",
 22: "Chaco",
 26: "Chubut",
 30: "Entre Ríos",
 34: "Formosa",
 38: "Jujuy",
 42: "La Pampa",
 46: "La Rioja",
 50: "Mendoza",
 54: "Misiones",
 58: "Neuquén",
 62: "Río Negro",
 66: "Salta",
 70: "San Juan",
 74: "San Luis",
 78: "Santa Cruz",
 82: "Santa Fe",
 86: "Santiago del Estero",
 90: "Tucumán",
 94: "TdF"}

In [27]:
empleo.index = empleo.index.map(dict_cod_provincia)
desempleo.index = desempleo.index.map(dict_cod_provincia)
empleo_asalariado_con_descuento.index = empleo_asalariado_con_descuento.index.map(dict_cod_provincia)
horas_remuneradas.index = horas_remuneradas.index.map(dict_cod_provincia)

### Cargo cosas para plots

In [28]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [29]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

### Analisis consistencia T2
ESTADO == 3

In [30]:
# Inactividad
def inactividad(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 3].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 3].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Inactividad'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [31]:
tasa, size, ratio, error = inactividad(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
ratio_inactividad = pd.concat([tasa, size, ratio, error], axis=1)
ratio_inactividad

,Varon,Mujer,N_v,N_m,Inactividad,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,15.764308,24.910792,131,228,63.283043,54.889256,71.676830,8.393787,13.263880
6,17.212656,37.113052,1014,2193,46.378982,43.298920,49.459043,3.080062,6.641073
10,26.625660,42.521526,359,645,62.616896,56.661974,68.571818,5.954922,9.510089
14,14.916865,38.001539,430,947,39.253318,34.682162,43.824474,4.571156,11.645273
18,23.454232,49.132665,322,654,47.736536,41.728354,53.744717,6.008181,12.586128
22,22.672399,53.108063,280,653,42.691067,36.469879,48.912255,6.221188,14.572577
26,22.534744,42.165967,384,718,53.442967,47.919343,58.966592,5.523625,10.335550
30,21.887972,42.574016,450,1010,51.411574,46.523433,56.299715,4.888141,9.507861
34,31.324823,66.728604,405,900,46.943620,41.477276,52.409965,5.466345,11.644489


In [91]:
# Create bar traces
bar_trace1 = go.Bar(
    x=empleo.index,
    y=desempleo['Búsqueda de trabajo'],
    name='Desempleo',
    marker_color=fundar_colores_primarios[0]
)

bar_trace2 = go.Bar(
    x=empleo.index,
    y=ratio_inactividad['Inactividad'],
    name='Inactividad',
    marker_color=fundar_colores_primarios[1]
)

# Create a layout
layout = go.Layout(
    title='Desempleo e Inactividad',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Create a Figure
fig = go.Figure(data=[bar_trace1, bar_trace2], layout=layout)

fig.add_shape(
    type='line',
    x0=empleo.index[0],
    y0=100,
    x1=empleo.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
#fig.update_yaxes(range=[0, 200])

# Save the plot as an HTML file
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'desempleo_e_inactividad.html'
plot(fig, filename=file_path + filename, auto_open=False)


'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/desempleo_e_inactividad.html'

### Analisis consistencia T3
PP07G2 == 1 Aguinaldo
PP07G4 == 1 OS

In [92]:
# T3 bis. 
def acceso_aguinaldo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07G2']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a aguinaldo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

def acceso_obra_social(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07G4']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a OS'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [93]:
tasa, size, ratio, error = acceso_aguinaldo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_aguinaldo = pd.concat([tasa, size, ratio, error], axis=1)
tasa, size, ratio, error = acceso_obra_social(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_obra_social = pd.concat([tasa, size, ratio, error], axis=1)

In [94]:
# Create bar traces
bar_trace1 = go.Bar(
    x=empleo.index,
    y=empleo_asalariado_con_descuento['Acceso a derechos laborales'],
    name='Descuento jubilatorio',
    marker_color=fundar_colores_primarios[0]
)

bar_trace2 = go.Bar(
    x=empleo.index,
    y=empleo_asalariado_con_aguinaldo['Acceso a aguinaldo'],
    name='Aguinaldo',
    marker_color=fundar_colores_primarios[1]
)

bar_trace3 = go.Bar(
    x=empleo.index,
    y=empleo_asalariado_con_obra_social['Acceso a OS'],
    name='Obra Social',
    marker_color=fundar_colores_primarios[2]
)


# Create a layout
layout = go.Layout(
    title='Accesos a derechos laborales',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Create a Figure
fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3], layout=layout)

fig.add_shape(
    type='line',
    x0=empleo.index[0],
    y0=100,
    x1=empleo.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)
fig.update_yaxes(range=[0, 150])

# Save the plot as an HTML file
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'consistencia_accesos_laborales.html'
plot(fig, filename=file_path + filename, auto_open=False)


'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/consistencia_accesos_laborales.html'

### Plot insercion laboral

In [122]:

# Define the error bars for each trace
error_y1 = dict(
    type='data',
    symmetric=False,
    array=empleo['LS'] - empleo['Actividad productiva'],
    arrayminus=empleo['Actividad productiva'] - empleo['LI'],
    visible=True,
    color=fundar_black,
    thickness=0.7
)

error_y2 = dict(
    type='data',
    symmetric=False,
    array=desempleo['LS'] - desempleo['Búsqueda de trabajo'],
    arrayminus=desempleo['Búsqueda de trabajo'] - desempleo['LI'],
    visible=True,
    color=fundar_black,
    thickness=0.7
)

error_y3 = dict(
    type='data',
    symmetric=False,
    array=empleo_asalariado_con_descuento['LS'] - empleo_asalariado_con_descuento['Acceso a derechos laborales'],
    arrayminus=empleo_asalariado_con_descuento['Acceso a derechos laborales'] - empleo_asalariado_con_descuento['LI'],
    visible=True,
    color=fundar_black,
    thickness=0.7
)

error_y4 = dict(
    type='data',
    symmetric=False,
    array=horas_remuneradas['LS'] - horas_remuneradas['Participacion trabajo remunerado'],
    arrayminus=horas_remuneradas['Participacion trabajo remunerado'] - horas_remuneradas['LI'],
    visible=True,
    color=fundar_black,
    thickness=0.7
)

# Create bar traces with error bars
bar_trace1 = go.Bar(
    x=empleo.index,
    y=empleo['Actividad productiva'],
    name='T1. Actividad productiva',
    marker_color=fundar_colores_primarios[0],
    error_y=error_y1
)

bar_trace2 = go.Bar(
    x=empleo.index,
    y=desempleo['Búsqueda de trabajo'],
    name='T2. Búsqueda de trabajo',
    marker_color=fundar_colores_primarios[1],
    error_y=error_y2
)

bar_trace3 = go.Bar(
    x=empleo.index,
    y=empleo_asalariado_con_descuento['Acceso a derechos laborales'],
    name='T3. Acceso a derechos laborales',
    marker_color=fundar_colores_primarios[2],
    error_y=error_y3
)

bar_trace4 = go.Bar(
    x=empleo.index,
    y=horas_remuneradas['Participacion trabajo remunerado'],
    name='T4. Participacion trabajo remunerado',
    marker_color=fundar_colores_primarios[3],
    error_y=error_y4
)


# Create scatter traces for the points
scatter_trace1 = go.Scatter(
    x=empleo.index,
    y=empleo['ER'],
    name='T1. ER',
    mode='markers',
    marker=dict(
        color=fundar_black,
        size=8
    ),
    yaxis='y2'
)

scatter_trace2 = go.Scatter(
    x=empleo.index,
    y=desempleo['ER'],
    name='T2. ER',
    mode='markers',
    marker=dict(
        color=fundar_black,
        size=8
    ),
    yaxis='y2'
)

scatter_trace3 = go.Scatter(
    x=empleo.index,
    y=empleo_asalariado_con_descuento['ER'],
    name='T3. ER',
    mode='markers',
    marker=dict(
        color=fundar_black,
        size=8
    ),
    yaxis='y2'
)

scatter_trace4 = go.Scatter(
    x=empleo.index,
    y=horas_remuneradas['ER'],
    name='T4. ER',
    mode='markers',
    marker=dict(
        color=fundar_black,
        size=8
    ),
    yaxis='y2'
)
# Create a layout
layout = go.Layout(
    title='Insercion laboral',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(y=1.1, orientation='h'),
    yaxis=dict(title='Actividad', range=[0, 170]),
    scattermode="group",
    yaxis2=dict(title='ER', overlaying='y', side='right', range=[0, 70])
)

fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3, bar_trace4,
                      scatter_trace1, scatter_trace2, scatter_trace3, scatter_trace4],
                layout=layout)

fig.add_shape(
    type='line',
    x0=empleo.index[0],
    y0=100,
    x1=empleo.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)

# Save the plot as an HTML file
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'actividad_productiva.html'
plot(fig, filename=file_path + filename, auto_open=False)

'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/actividad_productiva.html'